In [2]:
import gdown
import pandas as pd
import numpy as np

In [ ]:
# Variavel para escolher o escopo completo 2017-2024 Km 100 a 239 ou escopo reduzido 2023 Km 139 a 149
escopo = 'reduzido'  # 'completo' ou 'reduzido'
if escopo == 'completo':
    df_train = pd.read_csv("https://drive.google.com/uc?export=download&id=1ufAzTOoT0wo0kEgx2NY0JS2bR6_93BAW")
    url = f"https://drive.google.com/uc?id=1naPTeq_IgeXp9TM8wbojeF-zYpHJ_lPq"
    output = "df_test.csv"

    gdown.download(url, output, quiet=False)

    df_test = pd.read_csv(output)
else:
    df_train = pd.read_csv("data/df_train_trafego.csv")
    df_test = pd.read_csv("data/df_test_trafego.csv")

In [4]:
df_caracteristicas = pd.read_csv("https://drive.google.com/uc?export=download&id=12IEjF4wPAurDEhY7YrOyGe-APN9OM_Ls")

In [ ]:
def merge_acidente(df1, df2):
    df_train['datetime'] = pd.to_datetime(df_train['datetime'])
    df_test['datetime'] = pd.to_datetime(df_test['datetime'])
    df1 = df1.sort_values(by='km')
    df2 = df2.sort_values(by='km')
    df1 = pd.merge_asof(df1, df2, 
                           on='km', by=['sentido_via'], 
                           direction='nearest', tolerance=0.01)  
 
    return df1

In [6]:
df_train = merge_acidente(df_train, df_caracteristicas)
df_test = merge_acidente(df_test, df_caracteristicas)
df_train

,datetime,km,sentido_via,acidente,geometry,longitude,latitude,tipo_tracado,tipo_perfil_de_terreno,tipo_pavimento,tipo_de_uso_do_solo,numero_de_faixas,municipio,velocidade_regulamentada_veiculos_leves,velocidade_regulamentada_veiculos_pesados,marginal,iluminacao
0,2023-03-11 20:00:00,100.0,Decrescente,1.0,POINT (-48.688608 -26.757014),-48.688608,-26.757014,Tangente,Perfil Plano,Pavimento Flexível,Urbano,2,Balneário Piçarras,100,80,True,True
1,2019-12-21 00:00:00,100.0,Crescente,1.0,POINT (-48.688808 -26.757014),-48.688808,-26.757014,Tangente,Perfil Plano,Pavimento Flexível,Urbano,3,Balneário Piçarras,100,80,True,True
2,2017-07-27 23:00:00,100.0,Decrescente,0.0,POINT (-48.688608 -26.757014),-48.688608,-26.757014,Tangente,Perfil Plano,Pavimento Flexível,Urbano,2,Balneário Piçarras,100,80,True,True
3,2024-01-26 13:00:00,100.0,Decrescente,0.0,POINT (-48.688608 -26.757014),-48.688608,-26.757014,Tangente,Perfil Plano,Pavimento Flexível,Urbano,2,Balneário Piçarras,100,80,True,True
4,2019-01-13 18:00:00,100.0,Decrescente,1.0,POINT (-48.688608 -26.757014),-48.688608,-26.757014,Tangente,Perfil Plano,Pavimento Flexível,Urbano,2,Balneário Piçarras,100,80,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40939,2018-07-01 12:00:00,239.9,Decrescente,0.0,POINT (-48.643502 -27.857938),-48.643502,-27.857938,Tangente,Perfil Plano,Pavimento Flexível,Rural,2,Palhoça,110,90,False,False
40940,2019-04-17 04:00:00,239.9,Crescente,1.0,POINT (-48.643504 -27.857457),-48.643504,-27.857457,Tangente,Perfil Plano,Pavimento Flexível,Rural,2,Palhoça,110,90,False,False
40941,2017-02-28 19:00:00,239.9,Decrescente,0.0,POINT (-48.643502 -27.857938),-48.643502,-27.857938,Tangente,Perfil Plano,Pavimento Flexível,Rural,2,Palhoça,110,90,False,False
40942,2017-03-22 17:00:00,239.9,Decrescente,0.0,POINT (-48.643502 -27.857938),-48.643502,-27.857938,Tangente,Perfil Plano,Pavimento Flexível,Rural,2,Palhoça,110,90,False,False


In [ ]:
import holidays

def extrai_feature(df):
    # Criar colunas baseadas em datas

    df['mes'] = df['datetime'].dt.month
    df['dia'] = df['datetime'].dt.day
    df['hora'] = df['datetime'].dt.hour
    df['ano'] = df['datetime'].dt.year
    df['dia_da_semana'] = df['datetime'].dt.dayofweek

    # Fim de semana: sábado (5) ou domingo (6)
    df['fim_semana'] = df['dia_da_semana'].isin([5, 6])

    # Feriado nacional (Brasil)
    df['data_date'] = df['datetime'].dt.date
    feriados_br_data = holidays.Brazil(state='SC', years=df_train['datetime'].dt.year.unique())
    feriados_municipais = []

    from datetime import date
    for ano in df_train['datetime'].dt.year.unique():
        feriados_municipais.extend([
            date(ano, 2, 2),   # Navegantes
            date(ano, 3, 19),  # Penha, Gov. Celso Ramos, São José
            date(ano, 4, 21),  # Itapema
            date(ano, 4, 24),  # Palhoça
            date(ano, 5, 17),  # Biguaçu
            date(ano, 6, 13),  # Balneário Piçarras, Porto Belo
            date(ano, 6, 24),  # Tijucas
            date(ano, 7, 20),  # Balneário Camboriú
            date(ano, 9, 8),   # Itajaí
        ])
    # Transformando feriados da biblioteca holidays para objetos date
    feriados_br = [d for d in feriados_br_data]
    
    feriados = feriados_br + feriados_municipais
    df['feriado_br'] = df['data_date'].isin(feriados)

    return df

In [12]:
df_train = extrai_feature(df_train)
df_test = extrai_feature(df_test)

In [ ]:
df_train.to_csv("df_train.csv", index=False)
df_test.to_csv("df_test.csv", index=False)